In [ ]:
import pandas as pd
import re 
# here re means Regular Expression , so we are importing regular expression 

In [2]:
df=pd.read_csv('data_science_raw.csv')

In [4]:
df.head(5)
# # df.tail(5)

,Company,Company Score,Job Title,Location,Date,Salary,Skills
0,Razorpoint,3.4,Junior Data Scientist,"Manchester, England",3d,£35K (Employer est.),"Data mining, Big data, R, Data analysis skills..."
1,tower Hamlets,3.7,Assistant Data Scientist (Graduate) | R-2375,"London, England",5d,£31.00 Per Hour (Employer est.),"R, SQL, JavaScript, Python"
2,TW,4.0,Data Scientist,"Nottingham, England",30d+,£50K - £65K (Employer est.),"SQL, Maths, Data science, Python"
3,NatWest Group,4.6,Data Scientist,"Edinburgh, Scotland",2d,£41K - £54K (Glassdoor est.),"Software deployment, Data analysis skills, Sta..."
4,iwoca,3.9,Data Scientist - Ops,"London, England",7d,£60K - £90K (Employer est.),"MATLAB, R, Maths, C, Machine learning"


In [5]:
# first we saw we have k in salary , so we decided to replace k with 000 and also £ to ''
# df['Salary'].str.replace('£','')
# df['Salary'].str.replace('K','000')

# here i combine both statement
df['Salary']=df['Salary'].str.replace('£','').str.replace('K','000')
df

,Company,Company Score,Job Title,Location,Date,Salary,Skills
0,Razorpoint,3.4,Junior Data Scientist,"Manchester, England",3d,35000 (Employer est.),"Data mining, Big data, R, Data analysis skills..."
1,tower Hamlets,3.7,Assistant Data Scientist (Graduate) | R-2375,"London, England",5d,31.00 Per Hour (Employer est.),"R, SQL, JavaScript, Python"
2,TW,4.0,Data Scientist,"Nottingham, England",30d+,50000 - 65000 (Employer est.),"SQL, Maths, Data science, Python"
3,NatWest Group,4.6,Data Scientist,"Edinburgh, Scotland",2d,41000 - 54000 (Glassdoor est.),"Software deployment, Data analysis skills, Sta..."
4,iwoca,3.9,Data Scientist - Ops,"London, England",7d,60000 - 90000 (Employer est.),"MATLAB, R, Maths, C, Machine learning"
...,...,...,...,...,...,...,...
745,Deloitte,NaN,"Manager, AI Architect, Strategy, Governance & ...","London, England",30d+,NaN,NaN
746,Amazon Development Centre (London) Limited,NaN,"Sr. Applied Scientist AGI, Contextual Ads","London, England",30d+,NaN,NaN
747,sennder,NaN,(Senior) Artificial Intelligence Engineer,"London, England",30d+,NaN,NaN
748,Mott MacDonald,NaN,Senior Software Engineer / Machine Learning En...,"London, England",30d+,NaN,NaN


In [9]:
# In salary we also have this '(Employer est.)' and '(Glassdoor est.)' string 
# so we need to remove it or replace it

# r - This tells Python, "Treat this string exactly as it is written. Don't try to turn \n into a new line or \t into a tab." It’s best practice to use r whenever you are writing regex.

df['Salary']=df['Salary'].str.replace(r'\(Employer est.\)','',regex=True)
df['Salary']=df['Salary'].str.replace(r'\(Glassdoor est.\)','',regex=True)

In [14]:
df
# now we have 'per hour' also in our salary column ,normaly we have total of 40 hrs per week and in total we have 52 week so 52*40= 2080 , so we need to multiply salary with 2080 whereever we have per hour.
# we will create a function for this

def convert_to_year(salary):
    if 'Per Hour' in str(salary):
        hour_rate=re.findall(r'\d+\.?\d*',salary)
        if hour_rate:
            return float(hour_rate[0])*2080
    return salary

df['Salary']=df['Salary'].apply(convert_to_year)


In [22]:
# now we have salary range 50000-65000 we will split them into minimum and maximum salary , to do this first we need to convert those into string all values in column 

df['Salary']=df['Salary'].astype(str)


In [24]:
# now we will create 2 new column salary min and salary max to put those values 
# here we specify we are spiliting by - and using salary column 
df[['salary_min','salary_max']]=df['Salary'].str.split('-',expand=True)

In [27]:
# now to remove white spaces from those column , if present
df['salary_min']=df['salary_min'].str.strip()
df['salary_max']=df['salary_max'].str.strip()

In [30]:
# now converting those salary column back into numerical or integer value
df['salary_min']=pd.to_numeric(df['salary_min'],errors='coerce')
df['salary_max']=pd.to_numeric(df['salary_max'],errors='coerce')

In [34]:
# now we will split out location into city and country ,but before it make sure data is in string format
df['Location']=df['Location'].astype(str)

In [36]:
df[['city','country']]=df['Location'].str.split(',',n=1,expand=True)

In [ ]:
# now the only thing left is skill column - here we will split every skill as boolean column containing 0 and 1 
# here i have created a new dataframe with skills as column , for that i use str.get_dummies
skill_split=df['Skills'].str.get_dummies(sep=', ')

In [46]:
# now here am concatenating both my df and skill_split into one dataframe
df_with_skills=pd.concat([df,skill_split],axis=1)
df_with_skills.head(10)

,Company,Company Score,Job Title,Location,Date,Salary,Skills,salary_min,salary_max,city,...,Unreal Engine,Usability,Visual Studio,Warehouse experience,Waterfall,Web services,WordPress,XML,iOS,iOS development
0,Razorpoint,3.4,Junior Data Scientist,"Manchester, England",3d,35000,"Data mining, Big data, R, Data analysis skills...",35000.0,NaN,Manchester,...,0,0,0,0,0,0,0,0,0,0
1,tower Hamlets,3.7,Assistant Data Scientist (Graduate) | R-2375,"London, England",5d,64480.0,"R, SQL, JavaScript, Python",64480.0,NaN,London,...,0,0,0,0,0,0,0,0,0,0
2,TW,4.0,Data Scientist,"Nottingham, England",30d+,50000 - 65000,"SQL, Maths, Data science, Python",50000.0,65000.0,Nottingham,...,0,0,0,0,0,0,0,0,0,0
3,NatWest Group,4.6,Data Scientist,"Edinburgh, Scotland",2d,41000 - 54000,"Software deployment, Data analysis skills, Sta...",41000.0,54000.0,Edinburgh,...,0,0,0,0,0,0,0,0,0,0
4,iwoca,3.9,Data Scientist - Ops,"London, England",7d,60000 - 90000,"MATLAB, R, Maths, C, Machine learning",60000.0,90000.0,London,...,0,0,0,0,0,0,0,0,0,0
5,Spotify,3.8,"Data Scientist, Programming & Editorial","London, England",30d+,124000,"Data analysis skills, Tableau, SQL, Data scien...",124000.0,NaN,London,...,0,0,0,0,0,0,0,0,0,0
6,LSEG (London Stock Exchange Group),4.1,Data Scientist,"London, England",20d,80000,"Jira, MATLAB, Power BI, Operating systems, R",80000.0,NaN,London,...,0,0,0,0,0,0,0,0,0,0
7,CRED,4.1,Data Scientist,Remote,26d,35000 - 37000,"Power BI, Data mining, R, Data analysis skills...",35000.0,37000.0,Remote,...,0,0,0,0,0,0,0,0,0,0
8,MLN Security,3.2,Junior Data Scientist,"Barking, East of England, England",30d+,49000 - 70000,"TensorFlow, SAS, Big data, Java, AWS",49000.0,70000.0,Barking,...,0,0,0,0,0,0,0,0,0,0
9,PlayStation Global,4.0,Data Scientist,"London, England",11d,34000,"Tableau, SQL, Data management, Data science, C...",34000.0,NaN,London,...,0,0,0,0,0,0,0,0,0,0
